# BIL501 Project 

##### Halil Berk Dergi

In [3]:
import org.apache.spark._
import org.apache.spark.graphx._
// To make some of the examples work we will also need RDD
import org.apache.spark.rdd.RDD
import org.apache.spark.sql.functions._
import org.apache.spark.sql.types._
import org.apache.spark.sql.SparkSession
import org.apache.log4j.Logger
import org.apache.log4j.Level
import org.apache.spark.SparkContext
import org.apache.spark.SparkConf
import scala.util.MurmurHash

import org.apache.spark._
import org.apache.spark.graphx._
import org.apache.spark.rdd.RDD
import org.apache.spark.sql.functions._
import org.apache.spark.sql.types._
import org.apache.spark.sql.SparkSession
import org.apache.log4j.Logger
import org.apache.log4j.Level
import org.apache.spark.SparkContext
import org.apache.spark.SparkConf
import scala.util.MurmurHash


In [4]:
sc.stop() // stop sparkcontext

Creating a new sparksession where master is spark://Ubuntu-HBD:7077. There is a master and there are 3 workers in sparksession which is configured with STANDALONE MODE.

In [5]:
val spark = SparkSession.builder
     .master("spark://Ubuntu-HBD:7077")
     .appName("BIL501 Project")
     .getOrCreate()

spark: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@2f417ff7


In [6]:
sc.stop()

Reading textfile which is exported from forusquare API.

In [7]:
val venues = spark.read.textFile("export_venues.csv")
val links = spark.read.textFile("export_links.csv")

venues: org.apache.spark.sql.Dataset[String] = [value: string]
links: org.apache.spark.sql.Dataset[String] = [value: string]


In [8]:
venues.show(3)

+--------------------+
|               value|
+--------------------+
|4c0e115bc700c9b6c...|
|4bed1e076e8c20a19...|
|4ce9209e0f196dcbd...|
+--------------------+
only showing top 3 rows



In [9]:
links.show(3)

+--------------------+
|               value|
+--------------------+
|4c0e115bc700c9b6c...|
|4c0e115bc700c9b6c...|
|4c0e115bc700c9b6c...|
+--------------------+
only showing top 3 rows



Defining class Graph

In [10]:
class Graph[VD, ED] (val vertices: VertexRDD[VD], val edges: EdgeRDD[ED])

defined class Graph
Companions must be defined together; you may wish to use :paste mode for this.


Defining class Venues

In [11]:
case class Venues(Name:String,lat:Float,lng:Float)


defined class Venues


In [12]:
// the function used for creating venuesRDD.
def ayır(line:String):(Long,Venues)={
    // VertexID must be Long, so VenuesID is converted to Long from string.
    val a = MurmurHash.stringHash(line.split(",")(0).toString)     
    val b = line.split(",")(1).toString
    val c = line.split(",")(2).toFloat
    val d = line.split(",")(3).toFloat

    
    return (a, Venues(b,c,d))    
}


ayır: (line: String)(Long, Venues)


In [13]:
val venuesRDD: RDD[(VertexId, Venues)] =  venues.rdd.map(ayır)

venuesRDD: org.apache.spark.rdd.RDD[(org.apache.spark.graphx.VertexId, Venues)] = MapPartitionsRDD[11] at map at <console>:49


In [14]:
venuesRDD.take(3)

res4: Array[(org.apache.spark.graphx.VertexId, Venues)] = Array((914172510,Venues(Anıtkabir,39.926785,32.83659)), (-2086253975,Venues(Middle East Technical University (Orta Doğu Teknik Üniversitesi),39.90006,32.788715)), (919849682,Venues(HD İskender,39.95,32.830822)))


In [15]:
type Connection = Float
val linksRDD: RDD[Edge[Connection]] = links.rdd.map{ line => 
  val vs = line.split(",");
  Edge(MurmurHash.stringHash(vs(0).toString), MurmurHash.stringHash(vs(1).toString), vs(2).toFloat)
}

defined type alias Connection
linksRDD: org.apache.spark.rdd.RDD[org.apache.spark.graphx.Edge[Connection]] = MapPartitionsRDD[17] at map at <console>:46


### Creating VENUEs - GRAPH

In [16]:
val venuesgraph = Graph(venuesRDD, linksRDD)

venuesgraph: org.apache.spark.graphx.Graph[Venues,Connection] = org.apache.spark.graphx.impl.GraphImpl@150a88ad


Distances between edges

In [17]:
val facts: RDD[String] =
  venuesgraph.triplets.map(triplet =>
    triplet.srcAttr.Name + " ile " + triplet.dstAttr.Name+" arasındaki mesafe " + +triplet.attr + " m." )
facts.collect.foreach(println(_))

Starbucks ile City Hotel arasındaki mesafe 24.276623 m.
Starbucks ile 365 arasındaki mesafe 590.436 m.
Starbucks ile Liva Pastanesi arasındaki mesafe 754.1987 m.
Starbucks ile Seğmenler Park (Seğmenler Parkı) arasındaki mesafe 2321.4937 m.
Middle East Technical University (Orta Doğu Teknik Üniversitesi) ile ODTÜ Çarşı arasındaki mesafe 827.2606 m.
Middle East Technical University (Orta Doğu Teknik Üniversitesi) ile Kentpark arasındaki mesafe 1527.9105 m.
Middle East Technical University (Orta Doğu Teknik Üniversitesi) ile Susam arasındaki mesafe 842.96735 m.
Middle East Technical University (Orta Doğu Teknik Üniversitesi) ile Cepa arasındaki mesafe 1427.8602 m.
Middle East Technical University (Orta Doğu Teknik Üniversitesi) ile Çatı Cafe arasındaki mesafe 993.22015 m.
City Hotel ile Starbucks arasındaki mesafe 24.276623 m.
City Hotel ile 365 arasındaki mesafe 602.95807 m.
City Hotel ile Urfalı Hacı Mehmet Kebap arasındaki mesafe 82.51984 m.
City Hotel ile Adana Sofrası arasındaki mesa

Park Ehlikeyf ile Narquilla arasındaki mesafe 13.936407 m.
Park Ehlikeyf ile Pool Pub arasındaki mesafe 1455.6112 m.
Park Ehlikeyf ile Dürüm Aspava arasındaki mesafe 739.6461 m.
Park Ehlikeyf ile Gordion arasındaki mesafe 3457.0642 m.
Park Ehlikeyf ile Escape arasındaki mesafe 128.82419 m.
Köroğlu İşkembecisi ile Kızılay Square (Kızılay Meydanı) arasındaki mesafe 3499.7961 m.
Köroğlu İşkembecisi ile Starbucks arasındaki mesafe 244.10951 m.
Köroğlu İşkembecisi ile Pool Pub arasındaki mesafe 253.73813 m.
Köroğlu İşkembecisi ile Kahve Dünyası arasındaki mesafe 401.47513 m.
Köroğlu İşkembecisi ile Seğmenler Park (Seğmenler Parkı) arasındaki mesafe 1499.5859 m.
Hmbrgr - Homemade Burgers ile Cookshop arasındaki mesafe 144.82352 m.
Hmbrgr - Homemade Burgers ile Cinemaximum arasındaki mesafe 10.040905 m.
Hmbrgr - Homemade Burgers ile Tchibo arasındaki mesafe 157.36937 m.
Hmbrgr - Homemade Burgers ile Gordion arasındaki mesafe 116.1422 m.
Hmbrgr - Homemade Burgers ile Starbucks arasındaki mesaf

Next Level ile Big Chefs arasındaki mesafe 108.91422 m.
Next Level ile Armada arasındaki mesafe 371.19836 m.
Liva Pastanesi ile Taurus arasındaki mesafe 1848.3195 m.
Liva Pastanesi ile Marco Pascha arasındaki mesafe 463.06232 m.
Liva Pastanesi ile Kentpark arasındaki mesafe 2866.0266 m.
Liva Pastanesi ile Mado arasındaki mesafe 69.2479 m.
Liva Pastanesi ile Armada arasındaki mesafe 910.30896 m.
Barbella Cafe-Bar ile Kızılay Square (Kızılay Meydanı) arasındaki mesafe 250.9395 m.
Barbella Cafe-Bar ile Turuncu Cafe Pub arasındaki mesafe 78.92645 m.
Barbella Cafe-Bar ile Ezgi Cafe arasındaki mesafe 87.972565 m.
Barbella Cafe-Bar ile Mascha arasındaki mesafe 3.8639193 m.
Barbella Cafe-Bar ile Fox arasındaki mesafe 1.080477 m.
ANKAmall Teras ile Kızılay Square (Kızılay Meydanı) arasındaki mesafe 3844.701 m.
ANKAmall Teras ile Cinemaximum arasındaki mesafe 100.82916 m.
ANKAmall Teras ile Starbucks arasındaki mesafe 154.13858 m.
ANKAmall Teras ile Özsüt arasındaki mesafe 39.69572 m.
ANKAmall T

May 19 Stadium (19 Mayıs Stadyumu) ile ANKAmall arasındaki mesafe 1807.4233 m.
Doğata ile Park Center arasındaki mesafe 318.48584 m.
Doğata ile Kentpark arasındaki mesafe 3692.4602 m.
Doğata ile Atatürk Forest Farm (Atatürk Orman Çiftliği) arasındaki mesafe 30.29185 m.
Doğata ile Kahveci Hacıbaba arasındaki mesafe 4473.5176 m.
Doğata ile ANKAmall arasındaki mesafe 3331.505 m.
La Gioia Cafe Brasserie ile Nusr-Et Steakhouse arasındaki mesafe 283.63632 m.
La Gioia Cafe Brasserie ile Karum arasındaki mesafe 115.85084 m.
La Gioia Cafe Brasserie ile Louise Brasserie & Lounge arasındaki mesafe 881.4929 m.
La Gioia Cafe Brasserie ile Kuğulu Park arasındaki mesafe 244.94395 m.
La Gioia Cafe Brasserie ile Seğmenler Park (Seğmenler Parkı) arasındaki mesafe 486.05203 m.
Starbucks ile Cookshop arasındaki mesafe 26.80058 m.
Starbucks ile Hmbrgr - Homemade Burgers arasındaki mesafe 163.75366 m.
Starbucks ile Cinemaximum arasındaki mesafe 167.57681 m.
Starbucks ile Gordion arasındaki mesafe 51.42591 m

Mado ile Armada arasındaki mesafe 28.380136 m.
Starbucks ile Kentpark arasındaki mesafe 204.09789 m.
Starbucks ile Timboo Cafe arasındaki mesafe 243.87401 m.
Starbucks ile Bauhaus arasındaki mesafe 218.89427 m.
Starbucks ile Cepa arasındaki mesafe 29.113379 m.
Starbucks ile Cinemaximum arasındaki mesafe 29.981998 m.
Cinemaximum ile Kentpark arasındaki mesafe 178.65706 m.
Cinemaximum ile Starbucks arasındaki mesafe 227.53638 m.
Cinemaximum ile Timboo Cafe arasındaki mesafe 226.95999 m.
Cinemaximum ile Cepa arasındaki mesafe 17.09396 m.
Cinemaximum ile Starbucks arasındaki mesafe 29.981998 m.
Sunshine ile ODTÜ Tenis Kortları arasındaki mesafe 373.8777 m.
Sunshine ile BARikaT #1538 arasındaki mesafe 889.3394 m.
Sunshine ile Susam arasındaki mesafe 364.55045 m.
Sunshine ile ODTÜ Kocatepe Kahve Evi arasındaki mesafe 11.860792 m.
Sunshine ile Toolbar arasındaki mesafe 913.75134 m.
Big Chefs ile Brasserie Bomonti arasındaki mesafe 1119.0862 m.
Big Chefs ile Karum arasındaki mesafe 294.3255 m.

facts: org.apache.spark.rdd.RDD[String] = MapPartitionsRDD[35] at map at <console>:46


# PageRank

### 1) PartitioningStrategy = RandomVertexCut

In [30]:
venuesgraph.partitionBy(PartitionStrategy.RandomVertexCut)

res13: org.apache.spark.graphx.Graph[Venues,Connection] = org.apache.spark.graphx.impl.GraphImpl@2eede35


In [31]:
val ranks1 = venuesgraph.pageRank(0.0001).vertices

ranks1: org.apache.spark.graphx.VertexRDD[Double] = VertexRDDImpl[6182] at RDD at VertexRDD.scala:57


### 2) PartitioningStrategy = CanonicalRandomVertexCut

In [20]:
venuesgraph.partitionBy(PartitionStrategy.CanonicalRandomVertexCut)

res7: org.apache.spark.graphx.Graph[Venues,Connection] = org.apache.spark.graphx.impl.GraphImpl@2b2dbba8


In [21]:
val ranks2 = venuesgraph.pageRank(0.0001).vertices

ranks2: org.apache.spark.graphx.VertexRDD[Double] = VertexRDDImpl[2079] at RDD at VertexRDD.scala:57


### 3) PartitioningStrategy = EdgePartition1D

In [22]:
venuesgraph.partitionBy(PartitionStrategy.EdgePartition1D)

res8: org.apache.spark.graphx.Graph[Venues,Connection] = org.apache.spark.graphx.impl.GraphImpl@67bd06b7


In [23]:
val ranks3 = venuesgraph.pageRank(0.0001).vertices

ranks3: org.apache.spark.graphx.VertexRDD[Double] = VertexRDDImpl[3102] at RDD at VertexRDD.scala:57


### 4) PartitioningStrategy = EdgePartition2D

In [24]:
venuesgraph.partitionBy(PartitionStrategy.EdgePartition2D)

res9: org.apache.spark.graphx.Graph[Venues,Connection] = org.apache.spark.graphx.impl.GraphImpl@27235775


In [25]:
val ranks4 = venuesgraph.pageRank(0.0001).vertices

ranks3: org.apache.spark.graphx.VertexRDD[Double] = VertexRDDImpl[4125] at RDD at VertexRDD.scala:57


### 5) PartitioningStrategy = New 

In [28]:
venuesgraph.partitionBy(
  new PartitionStrategy {
    // select distinct sources only
    val capturedGraphData: Set[Long] = venuesgraph
      .edges
      .map(e => e.srcId)
      .distinct()
      .collect()
      .toSet

    override def getPartition(src: VertexId, dst: VertexId, numParts: PartitionID): PartitionID = {
      if(capturedGraphData.contains(dst)) 0
      else 1
    }
  }
)


res12: org.apache.spark.graphx.Graph[Venues,Connection] = org.apache.spark.graphx.impl.GraphImpl@2d1723d9


In [29]:
val ranks5 = venuesgraph.pageRank(0.0001).vertices

ranks5: org.apache.spark.graphx.VertexRDD[Double] = VertexRDDImpl[5159] at RDD at VertexRDD.scala:57


### Ranks by Venue's Name

In [26]:
val ranksByUsername = venuesRDD.join(EdgePartition2Dranks1).map{
  case (x,(y,z)) => (y.Name, z)
}
.sortBy(_._2,ascending=false)

println(ranksByUsername.collect().mkString("\n"))

(Kızılay Square (Kızılay Meydanı),13.599011290335826)
(ANKAmall,8.987377808294202)
(Kızılay AVM,6.220624864775678)
(Kahveci Hacıbaba,6.110454062983242)
(Güvenpark,5.9162318606960635)
(Ezgi Cafe,4.997005490396357)
(Seğmenler Park (Seğmenler Parkı),4.723452796375306)
(Kentpark,4.695524529050263)
(Kuğulu Park,4.184787966983499)
(Cepa,3.7242873091357063)
(Özsüt,3.568926305450556)
(Starbucks,3.568926305450556)
(Cinemaximum,3.568926305450556)
(Armada,3.47051621675922)
(Timboo Cafe,3.408143591428786)
(Starbucks,3.3908664922666434)
(IF Performance Hall,2.9872315463589922)
(Brasserie Bomonti,2.7468999206193314)
(Prestige Cinema,2.598862531486412)
(Passage,2.5238787103009153)
(Timboo Cafe,2.4575423267767196)
(Cinemaximum,2.375578341355078)
(Devrez,2.3198514379890938)
(Starbucks,2.313312289061767)
(HD İskender,2.30588720342222)
(Starbucks,2.234574316719174)
(Mado,2.121430792538547)
(Blackk Ankara,1.7319641717211292)
(Karum,1.7088269412695063)
(Leman Kültür,1.6376355436433265)
(Mickey's by Las Chi

ranksByUsername: org.apache.spark.rdd.RDD[(String, Double)] = MapPartitionsRDD[4138] at sortBy at <console>:50
